In [8]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.chat_models.openai import ChatOpenAI

__import__("pysqlite3")
import sys

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n", chunk_size=600, chunk_overlap=100
)

loader = UnstructuredFileLoader("./examples/chapter_one.md")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever()
)

chain.run("What does Winston live in?")

'The new context provides a deeper understanding of the oppressive and surveillance-filled environment in which Winston lives. It further emphasizes the constant monitoring and control imposed by the Party, with the presence of telescreens and posters reinforcing the idea that Big Brother is always watching. This heightened level of surveillance and lack of privacy in Winston\'s living space at Victory Mansions underscores the extreme measures taken by the Party to maintain power and suppress any form of dissent. Therefore, the refined answer would be:\n\nWinston lives in Victory Mansions, a cramped and dilapidated apartment complex provided by the Party in the dystopian society of George Orwell\'s novel "1984." The surveillance and control imposed by the Party, including telescreens and propaganda posters, create an oppressive atmosphere where every move and sound made by the inhabitants, including Winston, are monitored and scrutinized. In this environment, Winston\'s small alcove in